# Import Library

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as ec
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from datetime import date, timedelta
from bs4 import BeautifulSoup
import requests
import shutil
import os
import glob
import time,datetime
import pandas as pd
import numpy as np

# Declare values for variables

In [3]:
LOGIN_PATH = f"https://plm.wwwinc.com/login"
DOWNLOAD_PATH = "D:\\User\\nguyenta\\Wolverine World Wide\\New Asia Product Operations - Data - Data\\Data Warehouse\\Attachment"
PATH=f"https://plm.wwwinc.com/criticalPathCommercializationEdit/"
USERNAME = "tam.nguyen@wwwinc.com"
PASSWORD = "**********"
dt_string = datetime.datetime.now().strftime("%d.%m.%Y %H%M")
all_cpid=pd.read_csv(r'D:\OneDrive - Wolverine World Wide\Desktop\Comm CP ID.csv')
data = []

In [4]:
all_cpid

,Comm CP ID
0,67560


# Define Function to Get all Data in the table for each Comm ID include:
    File Name
    Category
    Size
    Created
    By
    Comm ID
    Link under tag "href"

In [7]:
def crawl_data(DOWNLOAD_PATH,LOGIN_PATH,all_cpid):
    print(f"{datetime.datetime.now()}-[get_report] start chrome object")
    # Start Chrome Driver ****************************************************************
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    options.add_experimental_option("prefs",
                                    {"download.default_directory": DOWNLOAD_PATH,
                                     "download.prompt_for_download": False,
                                     "download.directory_upgrade": True,
                                     "profile.default_content_settings.popups": 0,
                                     "safebrowsing.enabled": True
                                    }
                                    )
    # options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(), options=options)
    driver.implicitly_wait(300)
    driver.maximize_window()
    driver.get(LOGIN_PATH)
    time.sleep(5)
    # AUTOMATION ************************************************************************
    print(f"{datetime.datetime.now()}-[get_report] login")
    driver.find_element(By.NAME, "username").click()
    driver.find_element(By.NAME, "username").send_keys(USERNAME)
    driver.find_element(By.NAME, "password").click()
    driver.find_element(By.NAME, "password").send_keys(PASSWORD)
    time.sleep(3)
    driver.find_element(By.XPATH,"/html/body/div/div/div/div/div/div[3]/div[3]/div/div[5]/button").click()
    time.sleep(4)
    data=[]
    for cpid in all_cpid['Comm CP ID']:
        URL= f"{PATH}{cpid}"
        driver.get(URL)
        time.sleep(5)
        print(f"{datetime.datetime.now()}-[get_report] {cpid}")
        soup = BeautifulSoup(driver.page_source)
        table = soup.find('table', attrs={'class':'table table-striped table-bordered rowhover table-hove'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
    #     print(soup)
        try:
            for row in rows:
                content={}
                content["File"] = row.find_all('td')[1].text.strip()
                content["Category"] = row.find_all('td')[2].text.strip()
                content["Size"] = row.find_all('td')[3].text.strip()
                content["Created"] = row.find_all('td')[7].text.strip()
                content["By"] = row.find_all('td')[8].text.strip()
                content["CP_ID"]=cpid
                content["load_link"]=row.find('a').get('href')
                data.append(content)
        except:
            pass
    df = pd.DataFrame(data)
    df
    df.to_excel(f"output_CommCP {dt_string}.xlsx",index=False)
    print('Mission Completed!')
    data_new=pd.read_excel(f"output_CommCP {dt_string}.xlsx")
    print(data_new["File"])

In [8]:
if __name__=="__main__":
    crawl_data(DOWNLOAD_PATH,LOGIN_PATH,all_cpid)

2024-10-02 10:32:58.298413-[get_report] start chrome object
2024-10-02 10:33:11.290787-[get_report] login
2024-10-02 10:33:27.105015-[get_report] 67560
Mission Completed!
0    SS24 - T2 FIT FORM - AGILITY PEAK 5 BOA GTX  M...
1                           AGILITY PEAK 5 BOA GTX.rar
2                   RR_FW24_AGILITY PEAK 5 BOA GTX.zip
3      TR582394_J068061-AGILITY PEAK 5 BOA GTX M'S.pdf
4              T4 PASS AGILITY PEAK 5 BOA GTX M-W.xlsm
5                       MRS-1821 done luu trinh de.pdf
6                                M81705(VM463)done.pdf
7                                           M81705.pdf
8    SS24- RR- MERREL L- T2 & T3 Engineer Report- A...
Name: File, dtype: object
